<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


In [4]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])


Let's first import required Python packages for this lab:


In [5]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [6]:
import folium
import pandas as pd

In [7]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [13]:
## Task 1: Mark all launch sites on a map
import pandas as pd
import folium

# --- Step 1: Load CSV ---
spacex_df = pd.read_csv('spacex_launch_geo.csv')

# --- Step 2: Clean column names ---
spacex_df.columns = spacex_df.columns.str.strip()  # remove spaces

# --- Step 3: Detect relevant columns ---
# Launch site column
launch_col = next((c for c in spacex_df.columns if 'launch' in c.lower()), None)
if not launch_col:
    raise ValueError("Launch site column not found in CSV!")

# Latitude and Longitude columns
lat_col = next((c for c in spacex_df.columns if 'lat' in c.lower()), None)
lon_col = next((c for c in spacex_df.columns if 'lon' in c.lower()), None)
if not lat_col or not lon_col:
    raise ValueError("Latitude or Longitude column not found in CSV!")

# --- Step 4: Create base map ---
latitude = spacex_df[lat_col].mean()
longitude = spacex_df[lon_col].mean()
launch_map = folium.Map(location=[latitude, longitude], zoom_start=3)

# --- Step 5: Mark all unique launch sites ---
unique_sites = spacex_df[[launch_col, lat_col, lon_col]].drop_duplicates()
for i, row in unique_sites.iterrows():
    folium.Marker(
        location=[row[lat_col], row[lon_col]],
        popup=row[launch_col],
        icon=folium.Icon(color='red', icon='rocket', prefix='fa')
    ).add_to(launch_map)

# --- Step 6: Display the map ---
launch_map







First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [14]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [15]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [16]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [17]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [18]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
import pandas as pd
import folium
from folium.features import DivIcon

# --- Load CSV and clean columns ---
spacex_df = pd.read_csv('spacex_launch_geo.csv')
spacex_df.columns = spacex_df.columns.str.strip()

# Detect relevant columns
launch_col = next((c for c in spacex_df.columns if 'launch' in c.lower()), None)
lat_col = next((c for c in spacex_df.columns if 'lat' in c.lower()), None)
lon_col = next((c for c in spacex_df.columns if 'lon' in c.lower()), None)

# Base map
latitude = spacex_df[lat_col].mean()
longitude = spacex_df[lon_col].mean()
site_map = folium.Map(location=[latitude, longitude], zoom_start=3)

# --- Add a yellow circle near Houston ---
houston_coords = [29.7604, -95.3698]
houston_circle = folium.Circle(
    location=houston_coords,
    radius=5000,  # radius in meters
    color='#FFFF00',
    fill=True,
    fill_color='#FFFF00',
    fill_opacity=0.6
)
houston_marker = folium.map.Marker(
    houston_coords,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>Houston</b></div>',
    )
)
site_map.add_child(houston_circle)
site_map.add_child(houston_marker)

# --- Add circles and markers for each launch site ---
unique_sites = spacex_df[[launch_col, lat_col, lon_col]].drop_duplicates()
for i, row in unique_sites.iterrows():
    site_coords = [row[lat_col], row[lon_col]]
    
    # Circle around launch site
    folium.Circle(
        location=site_coords,
        radius=10000,  # 10 km radius
        color='#000000',
        fill=True,
        fill_color='#000000',
        fill_opacity=0.5
    ).add_child(
        folium.Popup(f"{row[launch_col]}")
    ).add_to(site_map)
    
    # Marker with label
    folium.map.Marker(
        location=site_coords,
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{row[launch_col]}</b></div>',
        )
    ).add_to(site_map)

# --- Display the map ---
site_map


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [20]:
# Task 2: Mark the success/failed launches for each site on the map
print(spacex_df.columns.tolist())
# Detect success/failure column
success_col = next((c for c in spacex_df.columns if 'class' in c.lower() or 'outcome' in c.lower()), None)
if not success_col:
    raise ValueError("Could not find a column indicating success/failure in the CSV!")

print(f"Using '{success_col}' as the success/failure column.")
from folium.plugins import MarkerCluster

marker_cluster = MarkerCluster().add_to(site_map)

for i, row in spacex_df.iterrows():
    site_coords = [row[lat_col], row[lon_col]]
    
    # Use detected success/failure column
    color = 'green' if row[success_col] == 1 else 'red'
    
    popup_text = f"""
    Site: {row[launch_col]}<br>
    Mission: {row.get('Mission', 'N/A')}<br>
    Outcome: {'Success' if row[success_col]==1 else 'Failure'}
    """
    
    folium.Marker(
        location=site_coords,
        popup=popup_text,
        icon=folium.Icon(color=color, icon='rocket', prefix='fa')
    ).add_to(marker_cluster)

site_map


['Flight Number', 'Date', 'Time (UTC)', 'Booster Version', 'Launch Site', 'Payload', 'Payload Mass (kg)', 'Orbit', 'Customer', 'Landing Outcome', 'class', 'Lat', 'Long']
Using 'Landing Outcome' as the success/failure column.


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [21]:
spacex_df.tail(10)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
46,43,2017-10-11,22:53:00,F9 FT B1031.2,KSC LC-39A,SES-11 / EchoStar 105,5200.00,GTO,SES EchoStar,Success (drone ship),1,28.573255,-80.646895
47,44,2017-10-30,19:34:00,F9 B4 B1042.1,KSC LC-39A,Koreasat 5A,3500.00,GTO,KT Corporation,Success (drone ship),1,28.573255,-80.646895
48,54,2018-05-11,20:14:00,F9 B5 B1046.1,KSC LC-39A,Bangabandhu-1,3600.00,GTO,Thales-Alenia/BTRC,Success (drone ship),1,28.573255,-80.646895
49,45,2017-12-15,15:36:00,F9 FT B1035.2,CCAFS SLC-40,SpaceX CRS-13,2205.00,LEO (ISS),NASA (CRS),Success (ground pad),1,28.563197,-80.576820
50,47,2018-01-08,1:00:00,F9 B4 B1043.1,CCAFS SLC-40,Zuma,3696.65,LEO,Northrop Grumman,Success (ground pad),1,28.563197,-80.576820
51,48,2018-01-31,21:25:00,F9 FT B1032.2,CCAFS SLC-40,GovSat-1 / SES-16,4230.00,GTO,SES,Controlled (ocean),0,28.563197,-80.576820
52,50,2018-03-06,5:33:00,F9 B4 B1044,CCAFS SLC-40,Hispasat 30W-6 PODSat,6092.00,GTO,Hispasat NovaWurks,No attempt,0,28.563197,-80.576820
53,52,2018-04-02,20:30:00,F9 B4 B1039.2,CCAFS SLC-40,SpaceX CRS-14,2647.00,LEO (ISS),NASA (CRS),No attempt,0,28.563197,-80.576820
54,53,2018-04-18,22:51:00,F9 B4 B1045.1,CCAFS SLC-40,Transiting Exoplanet Survey Satellite (TESS),362.00,HEO,NASA (LSP),Success (drone ship),1,28.563197,-80.576820
55,56,2018-06-04,4:45:00,F9 B4 B1040.2,CCAFS SLC-40,SES-12,5384.00,GTO,SES,No attempt,0,28.563197,-80.576820


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [22]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [23]:

# Detect success/failure column dynamically
success_col = next((c for c in spacex_df.columns if 'class' in c.lower() or 'outcome' in c.lower()), None)
if not success_col:
    raise ValueError("Could not find a column indicating success/failure in the CSV!")

# Create marker_color column
spacex_df['marker_color'] = spacex_df[success_col].apply(lambda x: 'green' if x == 1 else 'red')

# Verify
spacex_df[[launch_col, success_col, 'marker_color']].head()


,Launch Site,Landing Outcome,marker_color
0,CCAFS LC-40,Failure (parachute),red
1,CCAFS LC-40,Failure (parachute),red
2,CCAFS LC-40,No attempt,red
3,CCAFS LC-40,No attempt,red
4,CCAFS LC-40,No attempt,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


Your updated map may look like the following screenshots:


In [26]:
# Detect success/failure column dynamically
success_col = next((c for c in spacex_df.columns if 'class' in c.lower() or 'outcome' in c.lower()), None)
if not success_col:
    raise ValueError("Could not find a column indicating success/failure in the CSV!")

# Create marker_color column
spacex_df['marker_color'] = spacex_df[success_col].apply(lambda x: 'green' if x == 1 else 'red')

# Verify
spacex_df[[launch_col, success_col, 'marker_color']].head()


,Launch Site,Landing Outcome,marker_color
0,CCAFS LC-40,Failure (parachute),red
1,CCAFS LC-40,Failure (parachute),red
2,CCAFS LC-40,No attempt,red
3,CCAFS LC-40,No attempt,red
4,CCAFS LC-40,No attempt,red


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [28]:
import piplite
await piplite.install(['geopy'])


In [29]:
# TASK 3: Calculate the distances between a launch site to its proximities
from geopy.distance import geodesic

# Function to calculate distance in km between two coordinates
def distance_km(coord1, coord2):
    return geodesic(coord1, coord2).km
# Example coordinates (latitude, longitude)
features = {
    'city': [29.561, -95.083],       # e.g., Houston city near a launch site
    'railway': [28.572, -80.648],    # Example railway near Cape Canaveral
    'highway': [28.563, -80.567],    # Example highway
    'coastline': [28.573, -80.660]   # Example coastline
}
for i, row in unique_sites.iterrows():
    site_coords = [row[lat_col], row[lon_col]]
    
    # Draw a circle for launch site
    folium.Circle(
        location=site_coords,
        radius=10000,
        color='#000000',
        fill=True,
        fill_color='#000000',
        fill_opacity=0.5
    ).add_to(site_map)
    
    # Draw lines to nearby features
    for feature_name, feature_coords in features.items():
        dist = distance_km(site_coords, feature_coords)
        
        # Draw a line from site to feature
        folium.PolyLine(
            locations=[site_coords, feature_coords],
            color='blue',
            weight=2,
            opacity=0.6,
            tooltip=f"{feature_name} ({dist:.2f} km)"
        ).add_to(site_map)
        
        # Optional: add a small circle at feature
        folium.Circle(
            location=feature_coords,
            radius=500,
            color='orange',
            fill=True,
            fill_color='orange',
            fill_opacity=0.7
        ).add_to(site_map)
site_map


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [30]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [31]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [32]:
import math

def calculate_distance(lat1, lon1, lat2, lon2):
    """
    Calculate distance in kilometers between two coordinates using Haversine formula
    """
    R = 6371  # Earth radius in km
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    
    a = math.sin(delta_phi/2.0)**2 + \
        math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda/2.0)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    
    return R * c
# Launch site coordinate
launch_site_lat = 28.573255
launch_site_lon = -80.646895

# Closest coastline coordinate (you can adjust using MousePosition)
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print(f"Distance to coastline: {distance_coastline:.2f} KM")


Distance to coastline: 7.43 KM


In [33]:
from folium.features import DivIcon
import folium

coastline_coord = [coastline_lat, coastline_lon]

# Marker showing distance
distance_marker = folium.Marker(
    location=coastline_coord,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance_coastline),
    )
)

# Add marker and line to map
site_map.add_child(distance_marker)

folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], coastline_coord],
    color='blue',
    weight=2,
    opacity=0.6,
    tooltip=f"Distance: {distance_coastline:.2f} KM"
).add_to(site_map)

site_map


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [34]:
# Coordinates
launch_site_coord = [launch_site_lat, launch_site_lon]
coastline_coord = [coastline_lat, coastline_lon]

# Create a PolyLine connecting launch site to coastline
lines = folium.PolyLine(
    locations=[launch_site_coord, coastline_coord],
    color='blue',      # Line color
    weight=2,          # Line thickness
    opacity=0.6,       # Line transparency
    tooltip=f"Distance: {distance_coastline:.2f} KM"  # Shows distance on hover
)

# Add the line to the map
site_map.add_child(lines)

# Display map
site_map


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [35]:
# Launch site coordinate
launch_site_coord = [launch_site_lat, launch_site_lon]

# Coordinates of nearby features (replace these with actual coordinates from MousePosition)
city_coord = [29.561, -95.083]        # Houston city
railway_coord = [28.572, -80.648]     # Railway near site
highway_coord = [28.563, -80.567]     # Highway near site
coastline_coord = [28.56367, -80.57163]  # Closest coastline



In [36]:
city_dist = calculate_distance(launch_site_coord[0], launch_site_coord[1], city_coord[0], city_coord[1])
railway_dist = calculate_distance(launch_site_coord[0], launch_site_coord[1], railway_coord[0], railway_coord[1])
highway_dist = calculate_distance(launch_site_coord[0], launch_site_coord[1], highway_coord[0], highway_coord[1])
coastline_dist = calculate_distance(launch_site_coord[0], launch_site_coord[1], coastline_coord[0], coastline_coord[1])


In [40]:
# Launch site coordinate
launch_site_coord = [launch_site_lat, launch_site_lon]

# Coordinates of nearby features (replace these with actual coordinates from MousePosition)
city_coord = [29.561, -95.083]        # Houston city
railway_coord = [28.572, -80.648]     # Railway near site
highway_coord = [28.563, -80.567]     # Highway near site
coastline_coord = [28.56367, -80.57163]  # Closest coastlinefrom folium.features import DivIcon
city_dist = calculate_distance(launch_site_coord[0], launch_site_coord[1], city_coord[0], city_coord[1])
railway_dist = calculate_distance(launch_site_coord[0], launch_site_coord[1], railway_coord[0], railway_coord[1])
highway_dist = calculate_distance(launch_site_coord[0], launch_site_coord[1], highway_coord[0], highway_coord[1])
coastline_dist = calculate_distance(launch_site_coord[0], launch_site_coord[1], coastline_coord[0], coastline_coord[1])

# Function to add a marker with distance label
def add_distance_marker(map_obj, coord, distance, label):
    marker = folium.Marker(
        location=coord,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12; color:#d35400;"><b>{label}: {distance:.2f} KM</b></div>'
        )
    )
    map_obj.add_child(marker)
add_distance_marker(site_map, city_coord, city_dist, 'City')
add_distance_marker(site_map, railway_coord, railway_dist, 'Railway')
add_distance_marker(site_map, highway_coord, highway_dist, 'Highway')
add_distance_marker(site_map, coastline_coord, coastline_dist, 'Coastline')
site_map  # This will render the map



After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
